In [ ]:
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from itertools import count
import numpy as np
import time

from calmutils.localization import refine_point_lsq, detect_dog
from calmutils.localization.util import sig_to_full_width_at_quantile, get_ellipse_params

from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse

import pandas as pd

%matplotlib inline
plt.rcParams['figure.figsize'] = [6,6]

In [ ]:
# init new dataframe for results
df = pd.DataFrame()
run_counter = count()

# detection threshold
thresh = 0.3
# estimated FWHM (in m!)
# NB: z, y, x - order
fwhm_estim = np.array([600, 30, 30]) * 10e-9

In [ ]:
    
spot = {
    'run_idx': 1,
    'time': pd.to_datetime(time.asctime())
}
df = df.append(spot, ignore_index=True)

spot = {
    'run_idx': 2,
    'a' : 2,
    'time': pd.to_datetime(time.asctime())
}
df = df.append(spot, ignore_index=True)


sig = 3.2
img = np.zeros((200, 200, 20))

img[23,45,5] = 1
img[25,111, 5] = 1
img[21,15, 5] = 1
img[78,75, 5] = 1
img[48,25, 5] = 1

imga = ndi.gaussian_filter(img, sig)


imgs = [imga, imga]

starttime = pd.to_datetime(time.asctime())

for (channel_idx, img) in enumerate(imgs):
    plt.figure('Channel {} localizations'.format(channel_idx))
    plt.imshow(np.apply_along_axis(np.max, 2, img))
    guess = detect_dog(img, 0.4, 3.2)
    print(guess)
    for g in guess:
        g2, (p, _) = refine_point_lsq(img, g, np.array([int(np.round(3.2 * np.sqrt(2*np.log(2))))] * 3))
        mu = list(g2)[:2]
        print(mu)
        cov = np.diag(p[2+len(img.shape):])
        e = Ellipse(list(reversed(mu)), *get_ellipse_params(cov**2), fill=None, color='red', linewidth=2)
        plt.gca().add_artist(e)
    plt.show()

df


In [ ]:

# TODO: get images -> imgs, drop singleton dimensions
# TODO: append image list?
# TODO: custom parameter dict

starttime = pd.to_datetime(time.asctime())
run_idx = next(run_counter)

for (channel_idx, img) in enumerate(imgs):
    

    plt.figure('Channel {} localizations'.format(channel_idx))
    is3d = len(img.shape)>2
    projection = np.apply_along_axis(np.max, 0, img) if is3d else img
    plt.imshow(projection)
    
    # TODO: get pixelsize, offset, fov
    
    guess = detect_dog(img, threshold=thresh, fwhm=fwhm_estim, pixsize=pix_size)
    print(guess)
    for g in guess:
        
        # TODO: better cut region, not hardcode
        # TODO: handle unfittable (None)
        # TODO: handle 2d, 3d
        # TODO: get cutsize from fwhm, pixsize
        
        g2, (p, _) = refine_point_lsq(img, g, np.array([int(np.round(3.2 * np.sqrt(2*np.log(2))))] * 3))
        mu = list(g2)[:2] # TODO: handle both 2d, 3d
        
        # TODO: pixelsize!
        
        cov = np.diag(p[2+len(img.shape):]) if not is3d else np.diag(p[2+len(img.shape):-1])
    
        e = Ellipse(list(reversed(mu)), *get_ellipse_params(cov**2), fill=None, color='red', linewidth=2)
        plt.gca().add_artist(e)
    plt.show()
    
# TODO: plot simple summary (timeseries of sigmas)

In [ ]:
# TODO: save table
# TODO: save images?

In [ ]:
f = plt.figure()
f.add_axes()
ax = f.get_axes()[0]
print(ax)
ax.plot([1, 2, 4], '*-')
ax.plot([2], '*-')

In [ ]:
d = {'a': 1}
d.update({'b': 2})
d